# DeepSeek-R1 for OpenVINO GenAI 單獨執行含 Gradio 介面範例(deepseek-r1_run.ipynb)

by Jack OmniXRI, 2025/03/24  

原始範例: https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/notebooks/deepseek-r1/deepseek-r1.ipynb  

原始範例全部執行(Restart Kernel and Run All Cells）後，預設會取得  DeepSeek-R1-Distill-Qwen-14B 模型， INT4 權重壓縮格式，預計下載約 15 GB，轉檔後會產生 7.81 GB 大小檔案。如想要使用小一點的模型，可採用手動單步執行方式完成。   

本範例為簡化版，執行前需先完整執行過原始範例，取得模型並轉換好 OpenVINO 所需 IR 格式。  

## 1. 設定工作參數

In [1]:
# 指定執行裝置 "CPU", "GPU", "NPU", "AUTO"
device = "GPU"

In [2]:
# 手動切換下拉式選單，取得模型相關參數，包括語系、模型名稱、權重壓縮格式
# 預設會指定 Chinese語系問題範例， DeepSeek-R1-Distill-Llama-8B 模型, INT4 權重壓縮格式
# 相關內容可參考 llm_config_run.py

from llm_config_run import get_llm_selection_widget

form, lang, model_id_widget, compression_variant, _ = get_llm_selection_widget(device=device)

# 顯示表單
form

Box(children=(Box(children=(Label(value='Language:'), Dropdown(index=1, options=('English', 'Chinese'), value=…

In [3]:
# 將下拉式選單內容複製到變數中
model_configuration = model_id_widget.value
model_id = model_id_widget.label
print(f"Selected model {model_id} with {compression_variant.value} compression")

Selected model DeepSeek-R1-Distill-Llama-8B with INT4 compression


In [4]:
# 指定 OpenVINO IR 格式模型(*.xml)、權重(*.bin) 路徑，假設對應模型已下載並轉檔完成

from pathlib import Path

model_path = f"{model_id}\\{compression_variant.value}_compressed_weights" 
model_dir = Path(model_path)

## 2. 使用 OpenVINO GenAI 實例化 LLM 流水線

In [5]:
import openvino_genai as ov_genai
import sys
import time

print(f"Loading model from {model_dir}\n")

t0 = time.perf_counter() # 取得開始時間

pipe = ov_genai.LLMPipeline(str(model_dir), device) # 建立 LLM 流水線
generation_config = ov_genai.GenerationConfig() # 設定基本參數
generation_config.max_new_tokens = 128 # 設定最大新詞元數量

t1 = time.perf_counter() # 取得結束時間
print(f"Model loading time: {(t1-t0):.4f} seconds\n") # 列出模型載入時間

Loading model from DeepSeek-R1-Distill-Llama-8B\INT4_compressed_weights



C:\openvino_env_py310\lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


Model loading time: 8.4824 seconds



## 3. 啟動聊天機器人人機介面

Gradio 人機介面設定可參考 gradio_helper_deepseek_run.py  

執行後可直接在欄位上操作，亦可開啟網址 http://127.0.0.1:7860 (http://localhost:7860) 使用瀏覽器頁面進行操作。  

操作時可點選下方範例句子，或手動輸入問題，繁中/簡中/英文或混合皆可，接著按下「Summit」即可開始產生思考過程（簡中）及答案。如果顯示一半就停了，可點擊進階選項「Advance Options」。將最大新詞元「Max New Tokens」調到最大值即可改善。  

如果不習慣簡體中文輸出，可輸入提示詞「以下內容請用繁體中文輸出」就可改善。  

In [ ]:
from gradio_helper_deepseek_run import make_demo

demo = make_demo(pipe, model_configuration, model_id, lang.value, device == "NPU")

try:
    demo.launch(debug=True)
except Exception:
    demo.launch(debug=True, share=True)
# If you are launching remotely, specify server_name and server_port
# EXAMPLE: `demo.launch(server_name='your server name', server_port='server port in int')`
# To learn more please refer to the Gradio docs: https://gradio.app/docs/

C:\openvino_env_py310\lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
